In [1]:
import pandas as pd

In [2]:
# Load Data from Disk

# Geographical Data
states_option = pd.read_excel("../Data/Raw/Geographic/AuszugGV2QAktuell.xlsx", sheet_name = "Onlineprodukt_Gemeinden")

# Covid Data
covid = pd.read_csv("../Data/Raw/Covid/Aktuell_Deutschland_SarsCov2_Infektionen.csv")

In [3]:
# Reduce Geographical Data

# Remove unnecessary Coloums
states_option = states_option.iloc[:,[2, 3, 4, 7]]

# Filter so only Lands will be left
states_option = states_option.loc[lambda df: df.iloc[:, [0]].notna().any(axis=1)
                                            & df.iloc[:, [1]].isna().any(axis=1)
                                            & df.iloc[:, [2]].isna().any(axis=1) 
                                            & df.iloc[:, [3]].notna().any(axis=1), :]

# Remove Header without Value
states_option.drop(states_option.head(1).index, inplace=True)

# Remove unnecessary Coloums so only ID and federal state are left
states_option = states_option.iloc[:,[0, 3]]

# Give Coloums Names
states_option.columns.values[0] = "Id"
states_option.columns.values[1] = "Bundesland"

# Define id values as Integer
states_option["Id"] = states_option["Id"].astype("Int64")

# Convert to a Dictionary of federal states (id -> state)
states_dictionary = states_option.set_index("Id").to_dict("dict")["Bundesland"]

# Cleanup
del states_option

In [4]:
# Declare Function to reduce the IdLandkreis to its Land-Value

def reduce_id_landkreis(id):
  result = id[::-1]                                             # Reverse so that only the Land is on the right
  result = [(result[i:i+3]) for i in range(0, len(result), 3)]  # Split the input groups of 3
  result = result[1]                                            # Tage the second group with the Land as contend
  result = result[::-1]                                         # Reverse the Land value again to get the original

  return result

In [5]:
# Remove unnecessary Coloums

covid.drop(columns=["Altersgruppe", "Geschlecht", "IstErkrankungsbeginn", "NeuerFall", "NeuerTodesfall", "NeuGenesen"], inplace=True)

In [6]:
# Map IdLandkreis to its written federal state name

covid["IdLandkreis"] = covid["IdLandkreis"].astype("string").map(reduce_id_landkreis).astype("Int64").map(states_dictionary)

In [7]:
# Aggregate data of all dates and federal states to get a sum of infections for each date

# Empty dictionary to hold the data
grouped_data_reportingdate = dict()
grouped_data_refdate = dict()
grouped_data_complete = dict()

group_by_federalstates = covid.groupby(["IdLandkreis"], group_keys=True)

# Aggregate each federal state once with the reportingdate and refdate
for key in group_by_federalstates.groups:
    group_of_federalstate = group_by_federalstates.get_group(key)

    group_by_federalstate_and_reportingdate = group_of_federalstate.groupby(["Meldedatum"], group_keys=True).agg({"IdLandkreis": "first", "Meldedatum": "first", "AnzahlFall": sum, "AnzahlTodesfall": sum, "AnzahlGenesen": sum})
    result_one_key = key + "_Meldedatum"
    grouped_data_reportingdate[result_one_key] = group_by_federalstate_and_reportingdate
    
    group_by_federalstate_and_refdate = group_of_federalstate.groupby(["Refdatum"], group_keys=True).agg({"IdLandkreis": "first", "Refdatum": "first", "AnzahlFall": sum, "AnzahlTodesfall": sum, "AnzahlGenesen": sum})
    result_two_key = key + "_Refdatum"
    grouped_data_refdate[result_two_key] = group_by_federalstate_and_refdate

# Create one entry each for the reportingdate and refdate holding all federal states inside 
grouped_data_reportingdate["Deutschland_Meldedatum"] = pd.concat(grouped_data_reportingdate.values(), axis=0)
grouped_data_refdate["Deutschland_Refdatum"] = pd.concat(grouped_data_refdate.values(), axis=0)

# Fill the complete dictionary with all values
grouped_data_complete.update(grouped_data_reportingdate)
grouped_data_complete.update(grouped_data_refdate)

# Cleanup
del grouped_data_reportingdate
del grouped_data_refdate
del result_one_key
del result_two_key
del key
del group_of_federalstate
del group_by_federalstates
del group_by_federalstate_and_reportingdate
del group_by_federalstate_and_refdate

In [8]:
# Normalize header names 

for key in grouped_data_complete:
    grouped_data_complete[key].columns.values[0] = "Bundesland"

In [9]:
# Create Output Folder for Covid if not allready present

import os

output_path = "../Data/Output/Covid"

if not os.path.exists(output_path):
   os.makedirs(output_path)

# Save Dataset to Disk

grouped_data_complete["Deutschland_Meldedatum"].to_csv(output_path + "/Deutschland_Meldedatum.csv", index=False)
grouped_data_complete["Deutschland_Refdatum"].to_csv(output_path + "/Deutschland_Refdatum.csv", index=False)


# If you want all Data as seperated csv-files uncomment the code below

# for key in grouped_data_complete:
#     grouped_data_complete[key].to_csv(output_path + "/" + key + ".csv", index=False)

In [10]:
# Example to Load the Data with the concrete Datatypes

covid_reportingdate_test = pd.read_csv(output_path + "/Deutschland_Meldedatum.csv",
                                        parse_dates = ["Meldedatum"],
                                        dtype = {"Bundesland": "string",
                                                "AnzahlFall": "Int64",
                                                "AnzahlTodesfall": "Int64",
                                                "AnzahlGenesen": "Int64"})

covid_refdate_test = pd.read_csv(output_path + "/Deutschland_Refdatum.csv", 
                                        parse_dates = ["Refdatum"], 
                                        dtype = {"Bundesland": "string",
                                                "AnzahlFall": "Int64",
                                                "AnzahlTodesfall": "Int64",
                                                "AnzahlGenesen": "Int64"})